# Friction Socks
## PH 211 Lab
### Feb 21, 2021
### Zachary Bochanski

This notebook documents the process of determine the velocity and friction of and object on the earths surface using kinematics. The velocity will be slow enough that air drag is neglected. The lab takes a look at the objects motion in two dimentions. After velocity is found, the second part of this notebook explores how to determine the friction force acting on the object. In this case the object subject to these tests is a pair of knitted socks. 

### Dependencies

The following packages are used in assiting with calculations and visualizing the data in this lab.
- `numpy`
- `matplotlib.pyplot`
- `numpy.polynomial`
- `scipy.optimize`, specifically `fsolve`

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import polynomial as ply
from scipy.optimize import fsolve

### Materials

- Knitted wool socks
- Scale
- Tape measure

### Concept of Operations/ Procedure

1. Collect and convert/transform data etc
2. Conduct stats analysis, variability
3. Fit curve
4. Compart fit to standard kinematic expression (assuming constant acceleration)
5. Plot ideal curve to fit
6. Is terminal velocity suggested?
7. Unknown height

### Data Collection

The procedure for this lab is a fairly straightforward process:

1. Gather materials.
2. Roll socks.
3. Slide socks off the table.
4. Use tape measure to find how far away from the counter they impacted.
5. Repeat until you are satisfied.

One list of data to record, the distance the object traveled after leaving the counter.

1. `sock_range = []` in meters

| Range (m): |  |
| ------| ------|
|1.28 m |1.24 m |
|1.28 m |1.27 m |
|1.27 m |1.24 m |
|1.19 m |1.22 m |
|1.24 m |1.30 m |
|1.22 m |1.27 m |
|1.27 m |1.24 m |
|1.22 m |1.22 m |
|1.19 m |1.24 m |
|1.35 m |1.17 m |
|1.24 m |

### Record Data

Breaking out the 25 foot tape measure, record the impact location from the counter edge for each launch of the sock. 

Each range is stored in list: 

- `projectile_range_raw`

Loop though and convert each measuremnt to meters, store in list:

- `sock_range`

For later the data includes counter height:

- `counter height`

In [40]:
# list for projectile range
projectile_range_raw = [50.5, 50.5, 50, 47, 49, 48, 50, 48, 47, 53, 49, 50, 49, 48, 51, 50, 49, 48, 49, 46, 49 ] # inches

# convert inches to meters
sock_range = []
for num in range(len(projectile_range_raw)):
    num = projectile_range_raw[num]/39.37
    sock_range.append(num)

# counter height
counter_height = .91 # meters

print("Range in meters of sock ball:")
print("-----------------------------------")
print(sock_range)
print("-----------------------------------")
print("Counter height", counter_height, "m")

Range in meters of sock ball:
-----------------------------------
[1.282702565405131, 1.282702565405131, 1.27000254000508, 1.1938023876047752, 1.2446024892049785, 1.219202438404877, 1.27000254000508, 1.219202438404877, 1.1938023876047752, 1.346202692405385, 1.2446024892049785, 1.27000254000508, 1.2446024892049785, 1.219202438404877, 1.2954025908051816, 1.27000254000508, 1.2446024892049785, 1.219202438404877, 1.2446024892049785, 1.1684023368046736, 1.2446024892049785]
-----------------------------------
Counter height 0.91 m


### Data characteristics

Calculate:

- `range_mean`
- `range_median`
- `range_std`
- `range_min`
- `range_max`

Calculate:

- `variation`

In [45]:
# Calculate data characteristics
range_mean = np.mean(sock_range)
range_median = np.median(sock_range)
range_std = np.std(sock_range)
range_min = np.min(sock_range)
range_max = np.max(sock_range)

# Calculate variation
range_variation = 100.*range_std/range_mean

# Display
print("--------------------------")
print("Range Mean", range_mean)
print("Range Median", range_median)
print("Range Std Dev", range_std)
print("Range Min", range_min)
print("Range Max", range_max)
print("--------------------------")
print("Variation", range_variation)

--------------------------
Range Mean 1.2470215416621313
Range Median 1.2446024892049785
Range Std Dev 0.03892156926280366
Range Min 1.1684023368046736
Range Max 1.346202692405385
--------------------------
Variation 3.121162543104575


![FBD_lab6](https://github.com/ztbochanski/physics-211/raw/main/images/Lab6FBD.jpg)